# How to Train Your Own Cone Detection and Key Points Detection Networks

![](https://user-images.githubusercontent.com/22118253/70957091-fe06a480-2042-11ea-8c06-0fcc549fc19a.png)

In this notebook, we will demonstrate 
- how to train your own YOLOv3-based traffic cone detection network and do inference on a video.
- how to train your own KeyPoints detection network and do inference on pictures of traffic cone.

**[Accurate Low Latency Visual Perception for Autonomous Racing: Challenges Mechanisms and Practical Solutions](https://github.com/mit-han-lab/once-for-all)** is an accurate low latency visual perception system introduced by Kieran Strobel, Sibo Zhu, Raphael Chang, and Skanda Koppula.

![](https://user-images.githubusercontent.com/22118253/70950893-e2de6980-202f-11ea-9a16-399579926ee5.gif)

## 1. Preparation
Let's first install all the required packages:

In [ ]:
! sudo apt install unzip
! cd CVC-YOLOv3 && sudo python3 setup.py install && cd ..

Now, let's build the Cone Detection dataset and the corresponding dataloader. 

In [ ]:
print("Downloading Training Dataset")
! wget https://storage.googleapis.com/mit-driverless-open-source/YOLO_Dataset.zip
! unzip YOLO_Dataset.zip
! mv YOLO_Dataset CVC-YOLOv3/dataset/ && cd CVC-YOLOv3/dataset/
print("Downloading YOLOv3 Sample Weights")
! cd CVC-YOLOv3/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/sample-yolov3.weights && cd ..
print("Downloading Training and Validation Label")
! cd CVC-YOLOv3/dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/all.csv && cd ../..
! cd CVC-YOLOv3/dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/train.csv && cd ../..
! cd CVC-YOLOv3/dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/validate.csv && cd ../..
! rm YOLO_Dataset.zip